**TITRE**

In [30]:
import math

class AbstractRule:
    
    def _set_grammar(self,gram):
        self._grammar = gram
        
    def valuation(self):
        raise NotImplementedError
        
    def count(self):
        raise NotImplementedError

        
#Exception triggered when a rule uses a literal not in the grammar
class UnknownLiteralError(Exception):
    
    def __init__(self,literal):
        self.literal = literal

        
class ConstructorRule(AbstractRule):
    
    def __init__(self, *args):
        self._parameters = args
        #print (self._parameters[0])
        #print (self._parameters[1])
        self._valuation = math.inf

    def valuation(self):
        return self._valuation
    
    #The Grammar is supposed non ambiguous
    #check if all literals used in the rule are parts of the grammar
    def _verif_rule(self):
        try:
            self._grammar[self._parameters[0]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[0])
        try:
            self._grammar[self._parameters[1]]
        except KeyError:
            raise UnknownLiteralError(self._parameters[1])

    #Return true if there was no update
    def _update_valuation(self):
            #print("Valuation : Constructor")
            #print("old :"+str(self._valuation))
            self._old_val = self._valuation
            self._valuation = self._calc_valuation()
            #print("new :"+str(self._valuation))
            #print(self._old_val == self._valuation)
            return (self._old_val == self._valuation)

        
class UnionRule(ConstructorRule):
    
    def __init__(self,fst,snd):
        #print ("union")
        super().__init__(fst,snd)
        
    def _calc_valuation(self):
        #print ("union")
        return min(self._grammar[self._parameters[0]].valuation(),
                   self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        #print (self._parameters[0])
        #print (self._parameters[1])
        #print (self._grammar[self._parameters[0]].count(i))
        #print (self._grammar[self._parameters[1]].count(i))
        return self._grammar[self._parameters[0]].count(i) + self._grammar[self._parameters[1]].count(i)
    
        
class ProductRule(ConstructorRule):
    
    def __init__(self,fst,snd,cons):
        #print("product")
        super().__init__(fst,snd)
        self._constructor = cons
        
    def _calc_valuation(self):
        #print("product")
        return (self._grammar[self._parameters[0]].valuation() +
               self._grammar[self._parameters[1]].valuation())
    
    def count(self, i):
        print("count : product")
        res = 0
        valN1 = self._grammar[self._parameters[0]].valuation()
        valN2 = self._grammar[self._parameters[1]].valuation()
        print (self._parameters[0])
        print (self._parameters[1])
        print("i :" + str(i))
        print("valN1 :" + str(valN1))
        print("valN2 :" + str(valN2))
        # WARNING
        # This might break some grammars (weird grammars with 2 different epsilon rules)
        # but avoid some infinite recursion (e.g. Dyck grammar)
        
        if i == 0 and valN1 == 0 and valN2 == 0:
            return 1
        for k in range (valN1, i-valN2+1):
            print("k : "+str(k))
            print("l : "+str(i-k))
            res += self._grammar[self._parameters[0]].count(k) * self._grammar[self._parameters[1]].count(i-k)
            print("res : "+str(res))
        return res

    
class ConstantRule(AbstractRule):
    
    def __init__(self,obj):
        self._object = obj
        
    #We chose to put _update_valuation here too
    #to avoid testing the presence of the function
    #every time we want to call it on a rule
    def _update_valuation(self):
        return True

    
class EpsilonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
    
    def valuation(self):
        return 0
    
    def count(self, i):
        if i != 0:
            return 0
        else :
            return 1
    
    
class SingletonRule(ConstantRule):
    
    def __init__(self,obj):
        super().__init__(obj)
        
    def valuation(self):
        return 1
    
    def count(self, i):
        if i != 1:
            return 0
        else :
            return 1

#Triggered when a non terminal does not generate anything
class CircularGrammarError(Exception):
    
    def __init__(self,grammar,name):
        self.name = name
        self.grammar = grammar
        
    
def init_grammar(gram):
    #set the grammar for all rules
    for rule in gram.values() :
        rule._set_grammar(gram)
        #verifies every non-constant rule (check if all literal used in the rules are in the grammar)
        if isinstance(rule, ConstructorRule):
            rule._verif_rule()

    #As long as there's a change we update again
    #(we chose to make _update_valuation available
    #to any rule)
    while not all(rule._update_valuation() for rule in gram.values()):
        pass
    for name,rule in gram.items() :
        if rule.valuation() == math.inf :
            raise CircularGrammarError(gram,name)

In [31]:
#Grammar definition for binary trees
treeGram = {"Tree" : UnionRule("Node","Leaf"),
            "Node" : ProductRule("Tree","Tree",
                                lambda a, b : Node(a, b)),
            "Leaf" : SingletonRule("Leaf")}
init_grammar(treeGram)

In [32]:
#tests for the binary trees grammar

#valuation

assert (treeGram["Tree"].valuation() == 1)
assert (treeGram["Node"].valuation() == 2)
assert (treeGram["Leaf"].valuation() == 1)

#count

assert (treeGram["Tree"].count(0) == 0)
assert (treeGram["Tree"].count(1) == 1)
assert (treeGram["Tree"].count(2) == 1)
assert (treeGram["Tree"].count(3) == 2)
assert (treeGram["Tree"].count(4) == 5)
assert (treeGram["Tree"].count(5) == 14)
assert (treeGram["Tree"].count(6) == 42)
assert (treeGram["Tree"].count(7) == 132)
assert (treeGram["Tree"].count(8) == 429)
assert (treeGram["Tree"].count(9) == 1430)

count : product
Tree
Tree
i :0
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
count : product
Tree
Tree
i :4
valN1 :1
valN2 :1
k : 1
l : 3
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
val

valN2 :1
res : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 3
k : 3
l : 1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 5
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 28
k : 5
l : 1
count : product
Tree
Tree
i :5

Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
res : 2
k : 2
l : 2
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 3
k : 3
l : 1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : pro

Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 5
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 14
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 42
res : 42
k : 2
l : 5
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :5
valN1 :1
valN2 :1
k : 1
l : 4
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :4
valN1 :1
valN2 :1
k : 1
l : 3
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tre

Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
res : 7
k : 3
l : 2
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : 

...

valN2 :1
res : 1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
res : 7
k : 3
l : 2
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN

l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :4
valN1 :1
valN2 :1
k : 1
l : 3
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
res : 2
k : 2
l : 2
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : p

count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
res : 7
k : 3
l : 2
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 9
k : 4
l : 1
count : product
Tree
Tree
i :4
valN1 :1
valN2 :1
k : 1
l : 3
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1

count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 3
k : 3
l : 1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 5
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 14
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 90
k

count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 1
k : 2
l : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 2
res : 2
k : 2
l : 2
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l : 1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
res : 1
res : 3
k : 3
l : 1
count : product
Tree
Tree
i :3
valN1 :1
valN2 :1
k : 1
l : 2
count : product
Tree
Tree
i :1
valN1 :1
valN2 :1
count : product
Tree
Tree
i :2
valN1 :1
valN2 :1
k : 1
l

In [7]:
#Grammar defintion for Fibonacci words
fiboGram = {"Fib" : UnionRule("Vide", "Cas1"),
            "Cas1" : UnionRule("CasAu", "Cas2"),
            "Cas2" : UnionRule("AtomB", "CasBAu"),
            "Vide" : EpsilonRule(""),
            "CasAu" : ProductRule("AtomA", "Fib","".join),
            "AtomA" : SingletonRule("A"),
            "AtomB" : SingletonRule("B"),
            "CasBAu" : ProductRule("AtomB", "CasAu","".join)}
init_grammar(fiboGram)

In [8]:
#tests for the fibonacci words grammar

#valuation

assert (fiboGram["Fib"].valuation() == 0)
assert (fiboGram["Cas1"].valuation() == 1)
assert (fiboGram["Cas2"].valuation() == 1)
assert (fiboGram["Vide"].valuation() == 0)
assert (fiboGram["CasAu"].valuation() == 1)
assert (fiboGram["AtomA"].valuation() == 1)
assert (fiboGram["AtomB"].valuation() == 1)
assert (fiboGram["CasBAu"].valuation() == 2)

In [9]:
#Grammar definition for all the words with A and B letters
abGram = {"Words" : UnionRule("Vide","Cas1"),
          "Cas1" : UnionRule("CasAu","CasBu"),
          "AtomA" : SingletonRule("A"),
          "AtomB" : SingletonRule("B"),
          "CasAu" : ProductRule("AtomA", "Words","".join),
          "CasBu" : ProductRule("AtomB", "Words", "".join),
          "Vide" : EpsilonRule(""),
          }
init_grammar(abGram)

In [10]:
# tests for the (a,b) words grammar

#valuation
#TODO


In [33]:
#Grammar definition for Dyck words
dyckGram = {"Dyck" : UnionRule("Vide","Cas1"),
            "Cas1" : UnionRule("CasG","CasDouble"),
            "Vide" : EpsilonRule(""),
            "AtomG" : SingletonRule("("),
            "AtomD" : SingletonRule(")"),
            "CasG" : ProductRule("AtomG","CasD", "".join),
            "CasD" : ProductRule("Dyck","AtomD", "".join),
            "CasDouble" : ProductRule("Dyck","Dyck", "".join),
            }
init_grammar(dyckGram)

In [35]:
# tests for the Dyck words grammar

#valuation

assert (dyckGram["Dyck"].valuation() == 0)
assert (dyckGram["Cas1"].valuation() == 0)
assert (dyckGram["Vide"].valuation() == 0)
assert (dyckGram["AtomG"].valuation() == 1)
assert (dyckGram["AtomD"].valuation() == 1)
assert (dyckGram["CasG"].valuation() == 2)
assert (dyckGram["CasD"].valuation() == 1)
assert (dyckGram["CasDouble"].valuation() == 0)

#count

assert (dyckGram["Dyck"].count(0) == 1)
assert (dyckGram["Dyck"].count(1) == 1)
assert (dyckGram["Dyck"].count(2) == 2)
assert (dyckGram["Dyck"].count(3) == 5)
assert (dyckGram["Dyck"].count(4) == 5)
assert (dyckGram["Dyck"].count(5) == 14)
assert (dyckGram["Dyck"].count(6) == 42)
assert (dyckGram["Dyck"].count(7) == 132)
assert (dyckGram["Dyck"].count(8) == 429)
assert (dyckGram["Dyck"].count(9) == 1430)

count : product
AtomG
CasD
i :0
valN1 :1
valN2 :1
count : product
Dyck
Dyck
i :0
valN1 :0
valN2 :0


AssertionError: 

In [17]:
#Grammar definition for the (A,B) words without 3 times the same letter in a row
ab3Gram = {"Words" : UnionRule("Vide","Cas1"),
              "Cas1" : UnionRule("CasA","CasB"),
              "CasA" : ProductRule("AtomA","CasA1","".join),
              "CasA1" : UnionRule("CasB","CasA2"),
              "CasA2" : UnionRule("CasAB","Vide"),
              "CasB" : ProductRule("AtomB","CasB1","".join),
              "CasB1" : UnionRule("CasA","CasB2"),
              "CasB2" : UnionRule("CasBA","Vide"),
              "CasAB" : ProductRule("AtomA","CasAB1","".join),
              "CasAB1" : UnionRule("CasB","Vide"),
              "CasBA" : ProductRule("AtomB","CasBA1","".join),
              "CasBA1" : UnionRule("CasA","Vide"),
              "Vide" : EpsilonRule(""),
              "AtomA" : SingletonRule("A"),
              "AtomB" : SingletonRule("B")
             }

init_grammar(ab3Gram)

In [18]:
# tests for the (A,B) words without 3 times the same letter in a row grammar

#valuation
assert (ab3Gram["Words"].valuation() == 0)
assert (ab3Gram["Cas1"].valuation() == 1)
assert (ab3Gram["CasA"].valuation() == 1)
assert (ab3Gram["CasA1"].valuation() == 0)
assert (ab3Gram["CasA2"].valuation() == 0)
assert (ab3Gram["CasB"].valuation() == 1)
assert (ab3Gram["CasB1"].valuation() == 0)
assert (ab3Gram["CasB2"].valuation() == 0)
assert (ab3Gram["CasAB"].valuation() == 1)
assert (ab3Gram["CasAB1"].valuation() == 0)
assert (ab3Gram["CasBA"].valuation() == 1)
assert (ab3Gram["CasBA1"].valuation() == 0)
assert (ab3Gram["Vide"].valuation() == 0)
assert (ab3Gram["AtomA"].valuation() == 1)
assert (ab3Gram["AtomB"].valuation() == 1)